In [8]:
from dotenv import load_dotenv
import nest_asyncio

nest_asyncio.apply()
load_dotenv()

True

In [9]:
from llama_index.core import SimpleDirectoryReader, Document
from llama_parse import LlamaParse
import os

# setup the parser
parser = LlamaParse(
    result_type="markdown"
)

In [10]:
company_files = {
    "4℃": ["4℃レポート2024.pdf"],
    "I H I": ["I H I 統 合 報 告 書 2 0 2 4.pdf"],
    "NISSAN": ["NISSANサステナビリティデータブック2024.pdf"]
}

# Loading and labeling documents
documents = []
# path to directory having all documents
pdf_directory = "./pdf"

for company, file_names in company_files.items():
    for file_name in file_names:
        file_path = os.path.join(pdf_directory, file_name)
        if not os.path.isfile(file_path):
            print(f"File {file_path} does not exist. Skipping.")
            continue
        # Use simpleDirectoryReader for individual files
        reader = SimpleDirectoryReader(input_files=[file_path], file_extractor={".pdf": parser})
        company_documents = reader.load_data()
        for doc in company_documents:
            doc.metadata["company"] = company
        documents.extend(company_documents)

Started parsing the file under job_id 9cf59ef2-09b1-4710-ac8c-f5e5cf40a682
Started parsing the file under job_id 71cf4da6-a06b-4c70-8d32-cf9d65caffd9
Started parsing the file under job_id 60143c7a-910b-45a7-af28-ddb23994873e
.........

In [11]:
documents

[Document(id_='617d0008-cd1e-482a-93fb-5d96f2179ce1', embedding=None, metadata={'file_path': 'pdf/4℃レポート2024.pdf', 'file_name': '4℃レポート2024.pdf', 'file_type': 'application/pdf', 'file_size': 8888590, 'creation_date': '2025-01-29', 'last_modified_date': '2025-01-28', 'company': '4℃'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='# 統 合レポート\n\n# ４℃\n\n# レポート 2024\n\nすべては、お客様の “笑顔” や “ときめき” のために', path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}'),
 Document(id_='1cf4e9e4-a2a5-4029-9407-8ce129835791', embedding=None, metadata={'file_path': 'pdf/4℃レ

In [13]:
from llama_index.core import VectorStoreIndex
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine

# Create the vector stor index with the labeled documents
vector_query_engine = VectorStoreIndex.from_documents(
    documents,
    use_async=True,
).as_query_engine()

### テストをしてみる

- 感想はつよつよじゃないか


In [14]:
query = "第7次中期経営計画のテーマは何ですか？"

vector_query_engine.query(query)

Response(response='Challenge for Future 未来への挑戦～2030年に向けて～', source_nodes=[NodeWithScore(node=TextNode(id_='7572201f-a552-45ac-a28f-40c68fa076ad', embedding=None, metadata={'file_path': 'pdf/4℃レポート2024.pdf', 'file_name': '4℃レポート2024.pdf', 'file_type': 'application/pdf', 'file_size': 8888590, 'creation_date': '2025-01-29', 'last_modified_date': '2025-01-28', 'company': '4℃'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='dfc16f14-46c3-4dcf-a5dd-cfb39c7a70a0', node_type='4', metadata={'file_path': 'pdf/4℃レポート2024.pdf', 'file_name': '4℃レポート2024.pdf', 'file_type': 'application/pdf', 'file_size': 8888590, 'creation_date': '2025-01-29', 'last_modified_date': '2025-01-28', 'company': '4℃'}, hash='8c753bc520c6ee13

In [15]:
query = "４℃ホールディングスグループの連結子会社数はいくつありますか？"
vector_query_engine.query(query)

Response(response='７社', source_nodes=[NodeWithScore(node=TextNode(id_='3d0838cc-7912-4c3e-8d30-091f4bf573ce', embedding=None, metadata={'file_path': 'pdf/4℃レポート2024.pdf', 'file_name': '4℃レポート2024.pdf', 'file_type': 'application/pdf', 'file_size': 8888590, 'creation_date': '2025-01-29', 'last_modified_date': '2025-01-28', 'company': '4℃'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='19b0d8fa-07ed-4344-8d6e-a9f1528e34d0', node_type='4', metadata={'file_path': 'pdf/4℃レポート2024.pdf', 'file_name': '4℃レポート2024.pdf', 'file_type': 'application/pdf', 'file_size': 8888590, 'creation_date': '2025-01-29', 'last_modified_date': '2025-01-28', 'company': '4℃'}, hash='c9ed8333f87988550dc469eeea4512505759b57ebadbd35fd66c

In [16]:
query = "４℃ホールディングスの2024年2月期の1株当たり年間配当金はいくらですか？"
vector_query_engine.query(query)

Response(response='The annual dividend per share for 4℃ Holdings for the fiscal year ending in February 2024 is 83.00円.', source_nodes=[NodeWithScore(node=TextNode(id_='a192fa22-9bdd-4bbd-8858-871675aab5b5', embedding=None, metadata={'file_path': 'pdf/4℃レポート2024.pdf', 'file_name': '4℃レポート2024.pdf', 'file_type': 'application/pdf', 'file_size': 8888590, 'creation_date': '2025-01-29', 'last_modified_date': '2025-01-28', 'company': '4℃'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='af9bc723-2075-46ff-ae6a-bc16d22f2892', node_type='4', metadata={'file_path': 'pdf/4℃レポート2024.pdf', 'file_name': '4℃レポート2024.pdf', 'file_type': 'application/pdf', 'file_size': 8888590, 'creation_date': '2025-01-29', 'last_modified

### Query Engineを使ってみる

In [ ]:
# Setup base query engine as a tool with metadata
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="financial query engine",
            description="Query engine for financial data of cement companies",
        ),
    ),
]

# Setup the sub-question query engine
query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=True,
)

説明を読む感じでは、これはLangChain側で実装してもいいかなって感じはします

https://medium.com/@thomas_reid/llamaparse-rag-beats-all-comers-60948c6cc0e4

この記事を参考にしたコードを作成する

Rerankingと複数のpdfを読み込めるかをテストしながら実装してみる

In [17]:
import nest_asyncio
nest_asyncio.apply()

In [19]:
from dotenv import load_dotenv

load_dotenv()

True

In [20]:
from llama_index.core import SimpleDirectoryReader, ServiceContext, VectorStoreIndex
from llama_index.llms.openai import OpenAI
from llama_parse import LlamaParse
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import MarkdownElementNodeParser
from llama_index.postprocessor.cohere_rerank import CohereRerank